In [ ]:
import argparse
import yaml
import os, sys
import time
import logging, logging.config

import pandas as pd
import geopandas as gpd
import rasterio

import stat
import math

from tqdm import tqdm

from misc_fct import test_crs

In [ ]:
with open('config.yaml') as fp:
    cfg = yaml.load(fp, Loader=yaml.FullLoader)['statistical_analysis.py']    #  [os.path.basename(__file__)]


# Defitions of the functions


# Definition of the constants
INPUT_FOLDER=cfg['input']['input_folder']
PROCESSED=cfg['processed']
PROCESSED_FOLDER=PROCESSED['processed_folder']
FINAL=cfg['final']
FINAL_FOLDER=FINAL['final_folder']

# Inputs
ROADS=PROCESSED_FOLDER + PROCESSED['input_files']['roads']
TILES_DIR=PROCESSED_FOLDER + PROCESSED['input_files']['images']
TILES_INFO=PROCESSED_FOLDER + PROCESSED['input_files']['tiles']

# Outputs
STATS_ROADS=PROCESSED_FOLDER + PROCESSED['output_files']['stats']
STATS_TYPE=FINAL_FOLDER+ FINAL['stats_by_type']

# Main

## Import data

In [ ]:
# Importation of the files
roads=gpd.read_file(ROADS)
tiles_info = gpd.read_file(TILES_INFO)

In [ ]:
print(roads.columns)
print(tiles_info.columns)
roads.shape[0]


## Data treatment

In [ ]:
print(roads[roads.is_valid==False].shape[0])          # to_file(PROCESSED_FOLDER + '/shapefiles_gpkg/test_invalid_geom.shp')

valid_roads=roads[roads.is_valid==True].drop(columns=['ERSTELLUNG', 'ERSTELLU_1', 'HERKUNFT', 'HERKUNFT_J', 'HERKUNFT_M','KUNSTBAUTE', 'WANDERWEGE',
       'VERKEHRSBE', 'BEFAHRBARK', 'EROEFFNUNG', 'STUFE', 'RICHTUNGSG', 'KREISEL', 'EIGENTUEME', 'VERKEHRS_1', 'NAME', 'TLM_STRASS', 'STRASSENNA', 
       'SHAPE_Leng', 'Width'])

## Trouver pourquoi certaines géométries ne sont plus valides après l'importation et corriger ça!



In [ ]:
valid_roads.to_crs(epsg=3857, inplace=True)
tiles_info.to_crs(epsg=3857, inplace=True)
test_crs(valid_roads.crs, tiles_info.crs)

print(valid_roads[valid_roads.is_valid==False].shape[0])

clipped_roads = gpd.clip(valid_roads, tiles_info).explode()

# clipped_roads=gpd.GeoDataFrame()
# for idx in tqdm(tiles_info.index, desc='Clipping roads'):

#     roads_to_tile = gpd.clip(roads_poly, tiles_info.loc[idx,'geometry']).explode()

#     clipped_roads=pd.concat([clipped_roads,roads_to_tile], ignore_index=True)


### Calcul et analyse des statistiques de zone pour les routes

#### PCA

### Calcul et analyses des statistiques par types

#### Boxplots